<a href="https://colab.research.google.com/github/luuleitner/dasIT/blob/main/beamform_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [ ]:
# install dasIT from the GitHub repository

!pip install git+https://github.com/luuleitner/dasIT

In [1]:
import os


In [6]:
# import 5 RF-data frames captured on a Verasonics Vantage 256 system*, using a GE-9LD transducer, and a CIRS generalpurpose ultrasound phantom.
#
# * Leitner et al. 2020, "Detection of Motor Endplates in Deep and Pennate Skeletal Muscles in-vivo using Ultrafast Ultrasound",
# 2020 IEEE International Ultrasonics Symposium (IUS).
#

rfdata_path = '/content/rfdata'

if os.path.exists(rfdata_path) == False:
  os.mkdir(rfdata_path)
  os.chdir(rfdata_path)
  !wget -i https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/CIRSphantom_GE9LD_VVantage/COLABdownload_url.txt

os.chdir(rfdata_path)

--2022-08-04 13:45:13--  https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/CIRSphantom_GE9LD_VVantage/COLABdownload_url.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337 [text/plain]
Saving to: ‘COLABdownload_url.txt’

COLABdownload_url.t 100%[===================>]     337  --.-KB/s    in 0s      

2022-08-04 13:45:14 (14.9 MB/s) - ‘COLABdownload_url.txt’ saved [337/337]

--2022-08-04 13:45:14--  https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/CIRSphantom_GE9LD_VVantage/CIRS_phantom.h5
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 31501608 (30M) [application/octet-stream]
Saving to: ‘CIRS_phantom.h5’

CIRS_phantom.h5     100%[=================